<a href="https://colab.research.google.com/github/LuisaBeccar/ODMexamen/blob/main/graficosx4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ODM IMAGINARIO
Pensemos qué huiera pasado si en vez de la asignacions de +5 puntos al PUNTAJE, se hubiera asignado +2 puntos al promedio de los estudiantes de universidades argentinas.<br>
Asi calcule PI = promedio_carrera + 2 + el 50% de la nota del examen. <br>
para cada especialidad armo su ORDEN_IMAGINARIO a partir de ese PI, y luego con el PI también armo un ORDEN_GLOBAL_IMAGINARIO.


In [ ]:
import pandas as pd
import numpy as np
import requests

In [ ]:
url = ("https://raw.githubusercontent.com/LuisaBeccar/ODMexamen/refs/heads/main/Base_ODM2025.csv")
di = pd.read_csv(url)
di.columns

Index(['DNI', 'NOMBRE', 'APELLIDO', 'SEXO', 'ORIGEN', 'UNI', 'TIPO_UNI',
       'PAIS_UNI', 'CIUDAD_UNI', 'lat', 'long', 'CLASE_UNI', 'FECHA_TITULO',
       'DIAS_DESDE_TITULO', 'PROMEDIO_CARRERA', 'ESPECIALIDAD', 'NOTA_EXAMEN',
       'COMPONENTE', 'PUNTAJE', 'PUNTAJE_CRUDO', 'ODM', 'ODM_CRUDO',
       'ODM_GLOBAL', 'ODM_GLOBAL_CRUDO'],
      dtype='object')

In [ ]:
di['PI'] = np.where(di['TIPO_UNI'] == 'N',    # wehere esta columna toma valor N: hacer esta cuenta, sino, hacer esta otra
                    (di['PROMEDIO_CARRERA'] + 2) + (di['NOTA_EXAMEN'] / 2),  # si es uni N le agrego 2 puntos al promedio
                    di['PROMEDIO_CARRERA'] + (di['NOTA_EXAMEN'] / 2))  # si es E queda el puntaje como se calculaba historicamente: promedio_carrera + el 50% de la nota del examen

di.columns  # veo que se agrego PI a las columnas del dataframe

Index(['DNI', 'NOMBRE', 'APELLIDO', 'SEXO', 'ORIGEN', 'UNI', 'TIPO_UNI',
       'PAIS_UNI', 'CIUDAD_UNI', 'lat', 'long', 'CLASE_UNI', 'FECHA_TITULO',
       'DIAS_DESDE_TITULO', 'PROMEDIO_CARRERA', 'ESPECIALIDAD', 'NOTA_EXAMEN',
       'COMPONENTE', 'PUNTAJE', 'PUNTAJE_CRUDO', 'ODM', 'ODM_CRUDO',
       'ODM_GLOBAL', 'ODM_GLOBAL_CRUDO', 'PI'],
      dtype='object')

In [ ]:
## ODM_IMAGINARIO: usando el PI par acada especialidad y luego la cascada de desempate
def ODM_imaginario(di):
    di = di.sort_values(
        by=['ESPECIALIDAD', 'PI', 'NOTA_EXAMEN', 'PROMEDIO_CARRERA', 'DNI'],
        ascending=[True, False, False, False, True]
    )
    di['ODM_IMAGINARIO'] = di.groupby('ESPECIALIDAD').cumcount() + 1
    return di

## ODM_GLOBAL_IMAGINARIO sin agrupar especialidades, PI (sin especialidades, por PI y luego la cascada de desempate)
def ODM_global_imaginario(di):
    di = di.sort_values(
        by=['PI', 'NOTA_EXAMEN', 'PROMEDIO_CARRERA', 'DNI'],
        ascending=[False, False, False, True]).reset_index(drop=True)
    di['ODM_GLOBAL_IMAGINARIO'] = di.index + 1
    return di

In [ ]:
di = ODM_imaginario(di) # se agrega la columna ODM_IMAGINARIO a di
di = ODM_global_imaginario(di) # se agrega la columna ODM_GLOBAL_IMAGINARIO a di
di.columns

Index(['DNI', 'NOMBRE', 'APELLIDO', 'SEXO', 'ORIGEN', 'UNI', 'TIPO_UNI',
       'PAIS_UNI', 'CIUDAD_UNI', 'lat', 'long', 'CLASE_UNI', 'FECHA_TITULO',
       'DIAS_DESDE_TITULO', 'PROMEDIO_CARRERA', 'ESPECIALIDAD', 'NOTA_EXAMEN',
       'COMPONENTE', 'PUNTAJE', 'PUNTAJE_CRUDO', 'ODM', 'ODM_CRUDO',
       'ODM_GLOBAL', 'ODM_GLOBAL_CRUDO', 'PI', 'ODM_IMAGINARIO',
       'ODM_GLOBAL_IMAGINARIO'],
      dtype='object')

In [ ]:
di.head(100)

,DNI,NOMBRE,APELLIDO,SEXO,ORIGEN,UNI,TIPO_UNI,PAIS_UNI,CIUDAD_UNI,lat,...,COMPONENTE,PUNTAJE,PUNTAJE_CRUDO,ODM,ODM_CRUDO,ODM_GLOBAL,ODM_GLOBAL_CRUDO,PI,ODM_IMAGINARIO,ODM_GLOBAL_IMAGINARIO
0,42011937,NATALIA BELEN,BARROS QUINTEROS,F,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,-34.6037,...,5,60.57,55.57,1,1,1,1,57.57,1,1
1,43418248,EUGENIA LOURDES,REJON COCUZZA,F,arg,UNIVERSIDAD NACIONAL DE CUYO,N,Argentina,Mendoza,-32.8895,...,5,60.19,55.19,1,1,2,2,57.19,1,2
2,41280789,MARIANO ANDRES,GATTI GOMEZ,M,arg,UNIVERSIDAD NACIONAL DE CORDOBA,N,Argentina,Cordoba,-31.4201,...,5,60.07,55.07,1,1,3,3,57.07,1,3
3,40993904,LAURA,DOLLER,F,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,-34.6037,...,5,60.02,55.02,1,1,4,4,57.02,1,4
4,42657933,CLARA,ROLLERI CHAHER,F,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,-34.6037,...,5,59.91,54.91,1,1,5,5,56.91,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,41604711,CANDELA,PETRONI,F,arg,UNIVERSIDAD NACIONAL DE ROSARIO,N,Argentina,Rosario,-32.9442,...,5,56.33,51.33,6,6,93,103,53.33,6,96
96,42102526,BRUNO,LORENZANO,M,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,-34.6037,...,5,56.32,51.32,7,7,94,104,53.32,7,97
97,41916511,CATALINA,JORIS,F,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,-34.6037,...,5,56.31,51.31,14,14,95,105,53.31,14,98
98,95851294,ANGIE KARINA,LI PUMA OVALLES,F,extr,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,-34.6037,...,5,56.31,51.31,15,15,96,106,53.31,15,99


In [ ]:
df = di  # llamo df a di para poder usar las funciones mas facilmente sin recambiar adentro df por di

import unicodedata
# mini limpieza de poner mayuscula y sacar acentos a especialidades
def ESPECIALIZAR(texto):
    """
    Transforma un texto a mayúsculas y elimina las tildes.
    """
    if isinstance(texto, str):
        # Normaliza el texto para separar los caracteres base de los acentos
        texto_sin_tildes = unicodedata.normalize('NFD', texto)
        # Codifica a ASCII (ignorando los acentos) y decodifica de vuelta a UTF-8
        return texto_sin_tildes.encode('ascii', 'ignore').decode('utf-8').upper()
    else:
        # Devuelve el valor original si no es un string (por ejemplo, NaN)
        return texto

# Aplica la función 'limpiar_texto' a la columna 'ESPECIALIDAD' y guarda  sobrescribe la existente
df['ESPECIALIDAD'] = df['ESPECIALIDAD'].apply(ESPECIALIZAR)
df.head(5)

,DNI,NOMBRE,APELLIDO,SEXO,ORIGEN,UNI,TIPO_UNI,PAIS_UNI,CIUDAD_UNI,lat,...,COMPONENTE,PUNTAJE,PUNTAJE_CRUDO,ODM,ODM_CRUDO,ODM_GLOBAL,ODM_GLOBAL_CRUDO,PI,ODM_IMAGINARIO,ODM_GLOBAL_IMAGINARIO
0,42011937,NATALIA BELEN,BARROS QUINTEROS,F,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,-34.6037,...,5,60.57,55.57,1,1,1,1,57.57,1,1
1,43418248,EUGENIA LOURDES,REJON COCUZZA,F,arg,UNIVERSIDAD NACIONAL DE CUYO,N,Argentina,Mendoza,-32.8895,...,5,60.19,55.19,1,1,2,2,57.19,1,2
2,41280789,MARIANO ANDRES,GATTI GOMEZ,M,arg,UNIVERSIDAD NACIONAL DE CORDOBA,N,Argentina,Cordoba,-31.4201,...,5,60.07,55.07,1,1,3,3,57.07,1,3
3,40993904,LAURA,DOLLER,F,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,-34.6037,...,5,60.02,55.02,1,1,4,4,57.02,1,4
4,42657933,CLARA,ROLLERI CHAHER,F,arg,UNIVERSIDAD DE BUENOS AIRES,N,Argentina,Buenos Aires,-34.6037,...,5,59.91,54.91,1,1,5,5,56.91,1,5


In [ ]:
# df.to_csv('ODM2025_imaginario.csv')
# subido a github

# Probar indicador y graficos en ODM IMAGINARIO

In [ ]:
%%capture
! pip install plotly nbformat pandas numpy requests
import pandas as pd
import numpy as np
import requests
import plotly.express as px
import json
import os
import plotly.graph_objects as go
import unicodedata

In [ ]:
#el csv creado en el archivo oferta_cargos y subido a github
esp_oferta = pd.read_csv("https://raw.githubusercontent.com/LuisaBeccar/ODMexamen/refs/heads/main/generar_data/oferta_cargos.csv")
esp_oferta  #ESPECIALIDAD, postulantes, oferta (cargos ofrecidos)

,ESPECIALIDAD,postulantes,oferta
0,CIRUGIA GENERAL,668,150
1,NEUMONOLOGIA,38,32
2,INFECTOLOGIA,38,30
3,EMERGENTOLOGIA,35,45
4,NEFROLOGIA,29,30
5,HEMATOLOGIA,26,25
6,FISIATRIA (MEDICINA FISICA Y REHABILITACION),14,14
7,GENETICA MEDICA,12,6
8,ORTOPEDIA Y TRAUMATOLOGIA INFANTIL,10,7
9,REUMATOLOGIA,9,14


## FUNCIONES

## funciones indicador y tabla

voy a querer comparar el movimiento de los postulantes en el odm imaginario comparado con el crudo

In [ ]:
#dentro de este colab, df es el dataframe con todos los datos de odm, odm_crudos y odm_imaginarios

def calcular_indicador_impacto_i(df_input):
    # 1. Manejo de casos límite: si el DataFrame está vacío, no hay cálculo posible.
    if df_input.empty:
        return 0

    # 2. Preparación de los datos: se crea una copia del DataFrame para trabajar con ella
    # y se le asigna un 'ID_POSTULANTE' basado en su índice original.
    df_base = df_input.copy()
    df_base['ID_POSTULANTE'] = df_base.index

    # 3. Creación de los escenarios de ordenamiento.
    # Se crean tres DataFrames, cada uno ordenado por un criterio distinto.

    # Escenario 'CRUDO'
    # # Se usa 'ODM_CRUDO' o 'ODM_GLOBAL_CRUDO' para ordenar.
    df_pos = df_base.sort_values('ODM_CRUDO' if 'ODM_CRUDO' in df_base.columns else 'ODM_GLOBAL_CRUDO').reset_index(drop=True)
    df_pos['Posicion_crudo'] = df_pos.index + 1 # Se añade la posición en este ranking.

    # Escenario 'ODM_IMAGINARIO' (Orden de mérito modificado)
        #Se usa 'ODM_IMAGINARIO' o 'ODM_GLOBAL_IMAGINARIO' para ordenar.
    df_odm_pos = df_base.sort_values('ODM_IMAGINARIO' if 'ODM_IMAGINARIO' in df_base.columns else 'ODM_GLOBAL_IMAGINARIO').reset_index(drop=True)
    df_odm_pos['Posicion_odm'] = df_odm_pos.index + 1 # Se añade la posición en este ranking de imaginarios

    # Escenario 'Extremo' (Orden de mérito en el peor caso para 'E')
    # Se ordena a los de tipo 'E' al final de la lista.
    df_extremo_pos = df_base.sort_values('TIPO_UNI', ascending=False).reset_index(drop=True)
    df_extremo_pos['Posicion_extremo'] = df_extremo_pos.index + 1 # Se añade la posición en este ranking.

    # 4. Unión de los DataFrames.
    # Se combinan las posiciones de cada postulante en los tres escenarios en un solo DataFrame.
    # Esto es crucial para poder comparar las posiciones de cada individuo.
    df_merged = pd.merge(df_pos[['ID_POSTULANTE', 'TIPO_UNI', 'Posicion_crudo']], df_odm_pos[['ID_POSTULANTE', 'Posicion_odm']], on='ID_POSTULANTE', how='inner')
    df_merged = pd.merge(df_merged, df_extremo_pos[['ID_POSTULANTE', 'Posicion_extremo']], on='ID_POSTULANTE', how='inner')

    # 5. Filtrado de los postulantes extranjeros.
    # El indicador se enfoca en el grupo 'E', por lo que solo se trabajan con sus datos.
    df_extranjeros = df_merged[df_merged['TIPO_UNI'] == 'E']
    if df_extranjeros.empty:
        return 0

    # 6. Cálculo del indicador.
    # El indicador se basa en el desplazamiento promedio del grupo 'E'.

    # Posición promedio del grupo 'E' en cada escenario
    posicion_promedio_E = df_extranjeros['Posicion_odm'].mean() # este es el imaginario
    posicion_promedio_E_crudo = df_extranjeros['Posicion_crudo'].mean()
    posicion_promedio_E_extremo = df_extranjeros['Posicion_extremo'].mean()

    # Desplazamiento real: cuánto se movió el grupo 'E' del ranking original al modificado.
    desplazamiento_real = posicion_promedio_E - posicion_promedio_E_crudo

    # Desplazamiento máximo: cuánto se movió el grupo 'E' del ranking original al peor caso posible.
    desplazamiento_maximo = posicion_promedio_E_extremo - posicion_promedio_E_crudo

    # Cálculo final del indicador como un porcentaje
    if desplazamiento_maximo > 0:
        indicador_cambio_i = (desplazamiento_real / desplazamiento_maximo) * 100
    else:
        indicador_cambio_i = 0

    return indicador_cambio_i

#--------------------------------------------------------------------------

def generar_tabla_impacto_i(df, esp_oferta):
    resultados = []
    oferta_global = esp_oferta['oferta'].sum()
    especialidades = df['ESPECIALIDAD'].unique()

    # 1. Bucle por especialidad
    # Se recorre cada especialidad para calcular sus métricas de forma individual.
    for especialidad in especialidades:
        df_esp = df[df['ESPECIALIDAD'] == especialidad].copy()

        # Se obtiene la oferta de cargos para la especialidad.
        try:
            oferta_value = esp_oferta.loc[esp_oferta['ESPECIALIDAD'] == especialidad, 'oferta'].iloc[0]
            oferta_value = int(oferta_value)
        except (IndexError, TypeError):
            oferta_value = None

        # Se cuentan los postulantes 'N' y 'E' en el total de la especialidad.
        conteo_total = df_esp['TIPO_UNI'].value_counts()
        n_total = conteo_total.get('N', 0)
        e_total = conteo_total.get('E', 0)

        # Se calcula el impacto total para toda la especialidad.
        impacto_total_i = calcular_indicador_impacto_i(df_esp)

        # 2. Manejo del caso acotado.
        # Se calcula el impacto solo para el grupo de postulantes que están dentro de la cantidad de cargos ofertados.
        impacto_acotado_i = None
        n_acotado = 0
        e_acotado = 0
        if oferta_value is not None:
            # Se filtra el DataFrame para incluir solo a los postulantes con ranking dentro de la oferta.
            df_esp_acotado = df_esp.loc[df_esp['ODM_IMAGINARIO'] <= oferta_value].copy()

            # Se cuenta el número de 'N' y 'E' en este grupo acotado.
            conteo_tipos_acotado = df_esp_acotado['TIPO_UNI'].value_counts()
            n_acotado = conteo_tipos_acotado.get('N', 0)
            e_acotado = conteo_tipos_acotado.get('E', 0)

            # Se calcula el impacto para este grupo acotado.
            impacto_acotado_i = calcular_indicador_impacto_i(df_esp_acotado)

        # 3. Almacenamiento de resultados por especialidad.
        # Se agregan todos los resultados a la lista 'resultados'.
        resultados.append({
            'especialidad': especialidad, 'postulantes': len(df_esp),
            'N_total': n_total, 'E_total': e_total,
            'cargos': oferta_value if oferta_value is not None else 'N/A',
            'impacto_total_I': f"{impacto_total_i:.2f}%", 'impacto_acotado_I': f"{impacto_acotado_i:.2f}%" if impacto_acotado_i is not None else 'N/A',
            'N_acotado_I': n_acotado, 'E_acotado_I': e_acotado
        })

    # 4. Cálculo de las métricas globales.
    # Se repite el proceso, pero para todos los postulantes, sin filtrar por especialidad.
    conteo_total_global = df['TIPO_UNI'].value_counts()
    n_total_global = conteo_total_global.get('N', 0)
    e_total_global = conteo_total_global.get('E', 0)
    impacto_total_global_i = calcular_indicador_impacto_i(df)

    # Se calcula el impacto acotado a nivel global, usando la suma de todos los cargos.
    df_global_acotado_i = df.loc[df['ODM_GLOBAL_IMAGINARIO'] <= oferta_global].copy()
    conteo_tipos_acotado_global_i = df_global_acotado_i['TIPO_UNI'].value_counts()
    n_acotado_global_i = conteo_tipos_acotado_global_i.get('N', 0)
    e_acotado_global_i = conteo_tipos_acotado_global_i.get('E', 0)
    impacto_acotado_global_i = calcular_indicador_impacto_i(df_global_acotado_i)

    # Se agregan los resultados globales a la lista.
    resultados.append({
        'especialidad': 'TOTAL', 'postulantes': len(df),
        'N_total': n_total_global, 'E_total': e_total_global,
        'cargos': int(oferta_global),
        'impacto_total_I': f"{impacto_total_global_i:.2f}%", 'impacto_acotado_I': f"{impacto_acotado_global_i:.2f}%",
        'N_acotado_I': n_acotado_global_i, 'E_acotado_I': e_acotado_global_i
    })

    # 5. Conversión a DataFrame y retorno
    # Se convierte la lista de resultados en un DataFrame final.
    df_resultados = pd.DataFrame(resultados)
    return df_resultados


#--------------------------------------------------------------------------

## Funciones de graficos especialidades

In [ ]:
# creo clasificacion de especialidades segun tienen muchos postlantes o pocos, para hacer los graficos mas a medida

# epecialidades de pocos postulante
esp_menor100 = esp_oferta[esp_oferta['postulantes']<151]
esp_chicas = esp_menor100['ESPECIALIDAD'].unique()
esp_chicas = esp_chicas.tolist()

# especialidades de muchos postulantes
esp_100 = esp_oferta[esp_oferta['postulantes']>=151]
esp_grandes = esp_100['ESPECIALIDAD'].unique()
esp_grandes = esp_grandes.tolist()


#-------------------------------------------------------------
def triple_acotada(especialidad, indicador_acotado):
    df_esp = df[df['ESPECIALIDAD'] == especialidad].copy()
    try:
        oferta_value = esp_oferta.loc[esp_oferta['ESPECIALIDAD'] == especialidad, 'oferta'].iloc[0]
        oferta_value = int(oferta_value)
    except IndexError:
        print(f"No se encontró el valor de 'oferta' para la especialidad: {especialidad}")
        oferta_value = None
    if oferta_value is not None:
        df_esp = df_esp.loc[df_esp['ODM_IMAGINARIO'] <= oferta_value].copy()
    postulantes_count_filtrado = len(df_esp)

    df_ext = df_esp.sort_values('TIPO_UNI', ascending=False).reset_index(drop=True)
    df_ext['Orden'] = np.arange(1, len(df_ext)+1)
    df_ext['Variable'] = 'EXTREMO'

    df_odm = df_esp.sort_values('ODM').reset_index(drop=True)
    df_odm['Orden'] = np.arange(1, len(df_odm)+1)
    df_odm['Variable'] = 'ODM'

    # ESTE ES EL IMAGINARIO, que deje llamado crudo pero deberia cambiarlo de aca para abajo en todo, el crudo se llama limpio
    df_crudo = df_esp.sort_values('ODM_IMAGINARIO').reset_index(drop=True)
    df_crudo['Orden'] = np.arange(1, len(df_crudo)+1)
    df_crudo['Variable'] = 'ODM_IMAGINARIO'

    #ACA EN REALIDAD AGREGAR EL CRUDO real que ahora llamare limpio
    df_limpio = df_esp.sort_values('ODM_CRUDO').reset_index(drop=True)
    df_limpio['Orden'] = np.arange(1, len(df_limpio)+1)
    df_limpio['Variable'] = 'ODM_CRUDO'

    df_long = pd.concat([df_limpio, df_crudo, df_odm, df_ext ], ignore_index=True)
    fig = px.bar(df_long, x=[1]*len(df_long), y='Orden', color='TIPO_UNI',
                 orientation='h', facet_col='Variable',
                 color_discrete_map={'N': 'lightskyblue', 'E': 'red'},
                 category_orders={'Variable':['ODM_CRUDO','ODM_IMAGINARIO','ODM','EXTREMO' ]}, # ya agrego el limpio adelante
                 labels={'Orden': 'Orden de mérito', 'TIPO_UNI': 'Tipo de universidad'},
                 height=400, width=800)

    fig.update_layout(annotations=[dict(text=f"", x=0.5, y=0, xref='paper', yref='paper', showarrow=False,
        font=dict(size=12), xanchor='center', yanchor='top')])
    fig.update_traces(marker_line_width=0, textfont_size=8)
    fig.update_yaxes(autorange='reversed', tickmode='array', tickvals=[1,25, 100, 250, 500])
    fig.update_yaxes(tickfont=dict(size=8))
    fig.update_layout(title={'text': f"{especialidad}", 'x': 0, 'xanchor': 'left',
                             'y': 0.95, 'yanchor': 'top', 'pad': {'b': 20, 't': 5}},
                      showlegend=False)
    nombres_y = 1.05

    fig.add_annotation(text="CRUDO", xref="paper", yref="paper", x=0.07, y=nombres_y, showarrow=False, font=dict(size=10))
    fig.add_annotation(text=" IMAGINARIO", xref="paper", yref="paper", x=0.3, y=nombres_y, showarrow=False, font=dict(size=10))
    fig.add_annotation(text="DEFINITIVO", xref="paper", yref="paper", x=0.62, y=nombres_y, showarrow=False, font=dict(size=10))
    fig.add_annotation(text="EXTREMO", xref="paper", yref="paper", x=0.91, y=nombres_y, showarrow=False, font=dict(size=10))

    fig.add_annotation(text=f"Scope en los primeros {int(postulantes_count_filtrado)} puestos (cargos disponibles)",
                       xref="paper", yref="paper", x=0.5, y=-0.08,
                       showarrow=False, font=dict(family="Arial", size=12, color="black"),
                       xanchor='center', yanchor='middle')

    fig.update_xaxes(title="", showticklabels=False)
    fig.update_traces(hoverinfo='skip', hovertemplate=None)
    fig.show()



#--------------------------------------------------------------------------
def chicas(especialidad, indicador_total):
    df_esp = df[df['ESPECIALIDAD'] == especialidad].copy()
    try:
        oferta_value = esp_oferta.loc[esp_oferta['ESPECIALIDAD'] == especialidad, 'oferta'].iloc[0]
        oferta_value = int(oferta_value)
    except IndexError:
        oferta_value = None
    postulantes_count = len(df_esp)
    df_ext = df_esp.sort_values('TIPO_UNI', ascending=False).reset_index(drop=True)
    df_ext['Orden'] = np.arange(1, len(df_ext)+1)
    df_ext['Variable'] = 'EXTREMO'
    df_odm = df_esp.sort_values('ODM').reset_index(drop=True)
    df_odm['Orden'] = np.arange(1, len(df_odm)+1)
    df_odm['Variable'] = 'ODM'
    df_crudo = df_esp.sort_values('ODM_IMAGINARIO').reset_index(drop=True)
    df_crudo['Orden'] = np.arange(1, len(df_crudo)+1)
    df_crudo['Variable'] = 'ODM_IMAGINARIO'
    #ACA EN REALIDAD AGREGAR EL CRUDO real que ahora llamare limpio
    df_limpio = df_esp.sort_values('ODM_CRUDO').reset_index(drop=True)
    df_limpio['Orden'] = np.arange(1, len(df_limpio)+1)
    df_limpio['Variable'] = 'ODM_CRUDO'

    df_long = pd.concat([df_limpio, df_crudo, df_odm, df_ext ], ignore_index=True)

    if oferta_value is not None:
        if oferta_value >= postulantes_count:
            linea_y_pos = len(df_ext) + 1
            linea_text = f"Oferta: {oferta_value} (Todos admitidos)"
        else:
            linea_y_pos = oferta_value
            linea_text = f"Oferta: {oferta_value}"
    else:
        linea_y_pos = None
        linea_text = None

    fig = px.bar(df_long, x=[1]*len(df_long), y='Orden', color='TIPO_UNI',
                 orientation='h', facet_col='Variable',
                 color_discrete_map={'N': 'lightskyblue', 'E': 'red'},
                 category_orders={'Variable':['ODM_CRUDO','ODM_IMAGINARIO','ODM','EXTREMO' ]}, # ya agrego el limpio adelante
                 labels={'Orden': 'Orden de mérito', 'TIPO_UNI': 'Tipo de universidad'},
                 height=400, width=500)
    fig.update_layout(annotations=[dict(text=f"{postulantes_count} postulantes", x=0.5, y=0, xref='paper', yref='paper',
        showarrow=False, font=dict(size=12), xanchor='center', yanchor='top')])
    fig.update_traces(marker_line_width=0, textfont_size=8)
    fig.update_yaxes(autorange='reversed', tickmode='array', tickvals=[1,25, 100, 250, 500])
    fig.update_yaxes(tickfont=dict(size=8))
    fig.update_layout(title={'text': f"{especialidad}", 'x': 0, 'xanchor': 'left',
                             'y': 0.95, 'yanchor': 'top', 'pad': {'b': 25, 't': 5}},
                      showlegend=False)
    nombres_y = 1.05
    fig.add_annotation(text="CRUDO", xref="paper", yref="paper", x=0.05, y=nombres_y, showarrow=False, font=dict(size=10))
    fig.add_annotation(text="IMAGINARIO", xref="paper", yref="paper", x=0.27, y=nombres_y, showarrow=False, font=dict(size=10))
    fig.add_annotation(text="ODM", xref="paper", yref="paper", x=0.65, y=nombres_y, showarrow=False, font=dict(size=10))
    fig.add_annotation(text="EXTREMO", xref="paper", yref="paper", x=0.96, y=nombres_y, showarrow=False, font=dict(size=10))
    if linea_y_pos is not None:
        fig.add_shape(type='line', xref='paper', yref='y', x0=0, y0=linea_y_pos, x1=1, y1=linea_y_pos,
                      line=dict(color='black', width=2, dash='dot'))
        fig.add_annotation(text=linea_text, x=1.0, y=linea_y_pos, xref="x3", yref="y3",
                           showarrow=False, xanchor="right", yanchor="bottom",
                           font=dict(size=10, color="black"), bgcolor="rgba(255,255,255,0.7)")

    fig.add_annotation(text=f"Reorden de todos los postulantes",
                       xref="paper", yref="paper", x=0.5, y=-0.1,
                       showarrow=False, font=dict(family="Arial", size=12, color="black"),
                       xanchor='center', yanchor='middle')
    fig.update_xaxes(title="", showticklabels=False)
    fig.update_traces(hoverinfo='skip', hovertemplate=None)
    fig.show()


#--------------------------------------------------------------------------
def grandes(especialidad, indicador_total):
    df_esp = df[df['ESPECIALIDAD'] == especialidad].copy()
    try:
        oferta_value = esp_oferta.loc[esp_oferta['ESPECIALIDAD'] == especialidad, 'oferta'].iloc[0]
        oferta_value = int(oferta_value)
    except IndexError:
        oferta_value = None
    postulantes_count = len(df_esp)
    df_ext = df_esp.sort_values('TIPO_UNI', ascending=False).reset_index(drop=True)
    df_ext['Orden'] = np.arange(1, len(df_ext)+1)
    df_ext['Variable'] = 'EXTREMO'
    df_odm = df_esp.sort_values('ODM').reset_index(drop=True)
    df_odm['Orden'] = np.arange(1, len(df_odm)+1)
    df_odm['Variable'] = 'ODM'
    df_crudo = df_esp.sort_values('ODM_IMAGINARIO').reset_index(drop=True)
    df_crudo['Orden'] = np.arange(1, len(df_crudo)+1)
    df_crudo['Variable'] = 'ODM_IMAGINARIO'
    #ACA EN REALIDAD AGREGAR EL CRUDO real que ahora llamare limpio
    df_limpio = df_esp.sort_values('ODM_CRUDO').reset_index(drop=True)
    df_limpio['Orden'] = np.arange(1, len(df_limpio)+1)
    df_limpio['Variable'] = 'ODM_CRUDO'

    df_long = pd.concat([df_limpio, df_crudo, df_odm, df_ext ], ignore_index=True)

    if oferta_value is not None:
        if oferta_value >= postulantes_count:
            linea_y_pos = len(df_ext) + 1
            linea_text = f"Oferta: {oferta_value} (Todos admitidos)"
        else:
            linea_y_pos = oferta_value
            linea_text = f"Oferta: {oferta_value}"
    else:
        linea_y_pos = None
        linea_text = None

    fig = px.bar(df_long, x=[1]*len(df_long), y='Orden', color='TIPO_UNI',
                 orientation='h', facet_col='Variable',
                 color_discrete_map={'N': 'lightskyblue', 'E': 'red'},
                 category_orders={'Variable':['ODM_CRUDO','ODM_IMAGINARIO','ODM','EXTREMO' ]}, # ya agrego el limpio adelante
                 labels={'Orden': 'Orden de mérito', 'TIPO_UNI': 'Tipo de universidad'},
                 height=600, width=500)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

    fig.update_layout(annotations=[dict(text=f"{postulantes_count} postulantes", x=0.5, y=0, xref='paper', yref='paper',
        showarrow=False, font=dict(size=12), xanchor='center', yanchor='top')])
    fig.update_traces(marker_line_width=0, textfont_size=8)
    fig.update_yaxes(autorange='reversed', tickmode='array', tickvals=[1,25, 100, 250, 500])
    fig.update_yaxes(tickfont=dict(size=8))
    fig.update_layout(title={'text': f"{especialidad}", 'x': 0, 'xanchor': 'left',
                             'y': 0.95, 'yanchor': 'top', 'pad': {'b': 25, 't': 5}},
                      showlegend=False)
    nombres_y = 1.029
    fig.add_annotation(text="CRUDO", xref="paper", yref="paper", x=0.05, y=nombres_y, showarrow=False, font=dict(size=10))
    fig.add_annotation(text="IMAGINARIO", xref="paper", yref="paper", x=0.27, y=nombres_y, showarrow=False, font=dict(size=10))
    fig.add_annotation(text="ODM", xref="paper", yref="paper", x=0.65, y=nombres_y, showarrow=False, font=dict(size=10))
    fig.add_annotation(text="EXTREMO", xref="paper", yref="paper", x=0.96, y=nombres_y, showarrow=False, font=dict(size=10))
    if linea_y_pos is not None:
        fig.add_shape(type='line', xref='paper', yref='y', x0=0, y0=linea_y_pos, x1=1, y1=linea_y_pos,
                      line=dict(color='black', width=2, dash='dot'))
        fig.add_annotation(text=linea_text, x=1.0, y=linea_y_pos, xref="x3", yref="y3",
                           showarrow=False, xanchor="right", yanchor="bottom",
                           font=dict(size=10, color="black"), bgcolor="rgba(255,255,255,0.7)")

    fig.add_annotation(text=f"Reorden de todos los postulantes",
                       xref="paper", yref="paper", x=0.5, y=-0.1,
                       showarrow=False, font=dict(family="Arial", size=12, color="black"),
                       xanchor='center', yanchor='middle')
    fig.update_xaxes(title="", showticklabels=False)
    fig.update_traces(hoverinfo='skip', hovertemplate=None)
    fig.show()


## Funciones de graficos globales

In [ ]:
#---------------------------------------
#GLOBALES

def graficar_global_i(df, esp_oferta):
    oferta_global = esp_oferta['oferta'].sum()
    df_esp = df.copy()
    postulantes_count = len(df_esp)

    # Preparar DataFrames ordenados
    df_ext = df_esp.sort_values('TIPO_UNI', ascending=False).reset_index(drop=True)
    df_ext['Orden'] = np.arange(1, len(df_ext) + 1)
    df_ext['Variable'] = 'EXTREMO'

    df_odm = df_esp.sort_values('ODM_GLOBAL').reset_index(drop=True)
    df_odm['Orden'] = np.arange(1, len(df_odm) + 1)
    df_odm['Variable'] = 'ODM_GLOBAL'

    df_imaginario = df_esp.sort_values('ODM_GLOBAL_IMAGINARIO').reset_index(drop=True)
    df_imaginario['Orden'] = np.arange(1, len(df_imaginario) + 1)
    df_imaginario['Variable'] = 'ODM_GLOBAL_IMAGINARIO'

    df_crudo = df_esp.sort_values('ODM_CRUDO' if 'ODM_CRUDO' in df_esp.columns else 'ODM_GLOBAL_CRUDO').reset_index(drop=True)
    df_crudo['Orden'] = np.arange(1, len(df_crudo) + 1)
    df_crudo['Variable'] = 'GLOBAL_CRUDO'

    df_long = pd.concat([df_crudo, df_imaginario, df_odm, df_ext], ignore_index=True)

    # Posición línea oferta
    if oferta_global >= postulantes_count:
        linea_y_pos = len(df_ext) + 1
        linea_text = f"Oferta: {oferta_global} (Todos admitidos)"
    else:
        linea_y_pos = oferta_global
        linea_text = f"Oferta: {oferta_global}"

    indicador_cambio = calcular_indicador_impacto_i(df_esp)

    fig = px.bar(df_long,
                 x=[1] * len(df_long),
                 y='Orden',
                 color='TIPO_UNI',
                 orientation='h',
                 facet_col='Variable',
                 color_discrete_map={'N': 'lightskyblue', 'E': 'red'},
                 category_orders={'Variable': ['GLOBAL_CRUDO','ODM_GLOBAL_IMAGINARIO', 'ODM_GLOBAL', 'EXTREMO']},
                 labels={'Orden': 'Orden de mérito', 'TIPO_UNI': 'Tipo de universidad'},
                 height=3000,
                 width=1500)

    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))

    fig.update_layout(annotations=[dict(
        text=f"{postulantes_count} postulantes",
        x=0.5, y=0, xref='paper', yref='paper', showarrow=False,
        font=dict(size=12), xanchor='center', yanchor='top')])

    fig.update_traces(marker_line_width=0, textfont_size=8)
    fig.update_yaxes(autorange='reversed', tickmode='array', tickvals=[1, 150, 500, 1500, 3000, 6000])
    fig.update_yaxes(tickfont=dict(size=8))

    fig.update_layout(title={'text': "ORDEN DE MÉRITO GLOBAL",
                             'x': 0, 'xanchor': 'left',
                             'y': 1, 'yanchor': 'top',
                             'pad': {'b': 2, 't': 5}},
                     showlegend=True)

    titulitos_y = 1.009
    fig.add_annotation(text="CRUDO", xref="paper", yref="paper", x=0.08, y=titulitos_y, showarrow=False, font=dict(size=16))
    fig.add_annotation(text="IMAGINARIO", xref="paper", yref="paper", x=0.32, y=titulitos_y, showarrow=False, font=dict(size=16))
    fig.add_annotation(text="GLOBAL25", xref="paper", yref="paper", x=0.62, y=titulitos_y, showarrow=False, font=dict(size=18))
    fig.add_annotation(text="EXTREMO", xref="paper", yref="paper", x=0.9, y=titulitos_y, showarrow=False, font=dict(size=16))

    fig.update_xaxes(title="", showticklabels=False)

    # Línea oferta
    fig.add_shape(type='line',
                  xref='paper', yref='y',
                  x0=0, y0=linea_y_pos, x1=1, y1=linea_y_pos,
                  line=dict(color='black', width=2, dash='dot'))

    fig.add_annotation(text=linea_text,
                       x=1.0,
                       y=linea_y_pos,
                       xref="x3",
                       yref="y3",
                       showarrow=False,
                       xanchor="right",
                       yanchor="bottom",
                       font=dict(size=14, color="black"),
                       bgcolor="rgba(255,255,255,0.7)")

    fig.add_annotation(text=f"Reorden de todos los postulantes",
                       xref="paper", yref="paper", x=0.5, y=-0.01,
                       showarrow=False,
                       font=dict(family="Arial", size=12, color="black"),
                       xanchor='center',
                       yanchor='middle')

    fig.update_traces(hoverinfo='skip', hovertemplate=None)
    fig.show()

#-----------------

def graficar_global_acotado_i(df, esp_oferta):

    oferta_global = esp_oferta['oferta'].sum()
    df_esp = df[df['ODM_GLOBAL_IMAGINARIO'] <= oferta_global].copy()
    postulantes_count = len(df_esp)

    df_ext = df_esp.sort_values('TIPO_UNI', ascending=False).reset_index(drop=True)
    df_ext['Orden'] = np.arange(1, len(df_ext) + 1)
    df_ext['Variable'] = 'EXTREMO'

    df_odm = df_esp.sort_values('ODM_GLOBAL').reset_index(drop=True)
    df_odm['Orden'] = np.arange(1, len(df_odm) + 1)
    df_odm['Variable'] = 'ODM_GLOBAL'

    df_imaginario = df_esp.sort_values('ODM_GLOBAL_IMAGINARIO').reset_index(drop=True)
    df_imaginario['Orden'] = np.arange(1, len(df_imaginario) + 1)
    df_imaginario['Variable'] = 'ODM_GLOBAL_IMAGINARIO'

    df_crudo = df_esp.sort_values('ODM_CRUDO' if 'ODM_CRUDO' in df_esp.columns else 'ODM_GLOBAL_CRUDO').reset_index(drop=True)
    df_crudo['Orden'] = np.arange(1, len(df_crudo) + 1)
    df_crudo['Variable'] = 'GLOBAL_CRUDO'

    df_long = pd.concat([df_crudo, df_imaginario, df_odm, df_ext], ignore_index=True)

    indicador_cambio = calcular_indicador_impacto_i(df_esp)

    fig = px.bar(df_long,
                 x=[1] * len(df_long),
                 y='Orden',
                 color='TIPO_UNI',
                 orientation='h',
                 facet_col='Variable',
                 color_discrete_map={'N': 'lightskyblue', 'E': 'red'},
                 category_orders={'Variable': ['GLOBAL_CRUDO','ODM_GLOBAL_IMAGINARIO', 'ODM_GLOBAL', 'EXTREMO']},
                 labels={'Orden': 'Orden de mérito', 'TIPO_UNI': 'Tipo de universidad'},
                 height=2000,
                 width=1000)

    fig.update_traces(marker_line_width=0, textfont_size=8)
    fig.update_yaxes(autorange='reversed', tickmode='array', tickvals=[1, 150, 500, 1500, 3000, 6000])
    fig.update_yaxes(tickfont=dict(size=8))

    # Elimina los títulos de las facetas para evitar superposición
    fig.for_each_annotation(lambda a: a.update(text=""))

    # Actualiza el layout del título principal
    fig.update_layout(title={'text': "ORDEN DE MÉRITO GLOBAL ACOTADO",
                             'x': 0, 'xanchor': 'left',
                             'y': 1, 'yanchor': 'top',
                             'pad': {'b': 2, 't': 5}},
                     showlegend=True)

    # Agrega las anotaciones personalizadas para los títulos
    # Ajusta las coordenadas 'x' para que se centren mejor
    titulitos_y = 1.012
    fig.add_annotation(text="CRUDO", xref="paper", yref="paper", x=0.07, y=titulitos_y, showarrow=False, font=dict(size=16))
    fig.add_annotation(text="IMAGINARIO", xref="paper", yref="paper", x=0.375, y=titulitos_y, showarrow=False, font=dict(size=16))
    fig.add_annotation(text="GLOBAL25", xref="paper", yref="paper", x=0.625, y=titulitos_y, showarrow=False, font=dict(size=16))
    fig.add_annotation(text="EXTREMO", xref="paper", yref="paper", x=0.92, y=titulitos_y, showarrow=False, font=dict(size=16))

    fig.update_xaxes(title="", showticklabels=False)

    fig.add_annotation(text=f"Scope en los primeros {int(postulantes_count)} puestos (cargos disponibles)",
                       xref="paper", yref="paper", x=0.5, y=-0.01,
                       showarrow=False,
                       font=dict(family="Arial", size=12, color="black"),
                       xanchor='center',
                       yanchor='middle')

    fig.update_traces(hoverinfo='skip', hovertemplate=None)
    fig.show()

# EJECUCION de funciones

## TABLA

In [ ]:
# --- Bloque principal para generar los gráficos ---
# 1. Genera la tabla de impacto una sola vez
tabla_impacto_i = generar_tabla_impacto_i(df, esp_oferta) # en este notebook df es ODM2025_Imaginario (tiene ODM, CRUDO, IMAGINARIO)
tabla_impacto_i
#tabla_impacto.to_csv('tabla_impacto.csv', index=False)

,especialidad,postulantes,N_total,E_total,cargos,impacto_total_I,impacto_acotado_I,N_acotado_I,E_acotado_I
0,PEDIATRIA Y PEDIATRICAS ARTICULADAS,470,337,133,428,34.28%,36.13%,332,96
1,CLINICA MEDICA,285,211,74,267,26.62%,27.59%,208,59
2,NEUROCIRUGIA,170,120,50,23,32.12%,81.82%,22,1
3,CIRUGIA INFANTIL (CIRUGIA PEDIATRICA),54,47,7,8,46.77%,0.00%,8,0
4,TOCOGINECOLOGIA,476,373,103,170,34.41%,54.70%,165,5
5,CIRUGIA GENERAL,668,428,240,150,28.64%,42.22%,133,17
6,ANESTESIOLOGIA,591,510,81,111,29.62%,44.71%,110,1
7,GASTROENTEROLOGIA,125,65,60,30,25.55%,56.32%,22,8
8,ORTOPEDIA Y TRAUMATOLOGIA,464,213,251,140,28.66%,34.10%,104,36
9,CARDIOLOGIA,358,250,108,76,31.01%,48.70%,74,2


In [ ]:
corta_impacto_i = tabla_impacto_i[['especialidad','impacto_total_I',	'impacto_acotado_I',	'N_acotado_I',	'E_acotado_I' ]]
corta_impacto_i

,especialidad,impacto_total_I,impacto_acotado_I,N_acotado_I,E_acotado_I
0,PEDIATRIA Y PEDIATRICAS ARTICULADAS,34.28%,36.13%,332,96
1,CLINICA MEDICA,26.62%,27.59%,208,59
2,NEUROCIRUGIA,32.12%,81.82%,22,1
3,CIRUGIA INFANTIL (CIRUGIA PEDIATRICA),46.77%,0.00%,8,0
4,TOCOGINECOLOGIA,34.41%,54.70%,165,5
5,CIRUGIA GENERAL,28.64%,42.22%,133,17
6,ANESTESIOLOGIA,29.62%,44.71%,110,1
7,GASTROENTEROLOGIA,25.55%,56.32%,22,8
8,ORTOPEDIA Y TRAUMATOLOGIA,28.66%,34.10%,104,36
9,CARDIOLOGIA,31.01%,48.70%,74,2


### merge imaginaria impacto con impacto real

In [ ]:
url2 = ("https://raw.githubusercontent.com/LuisaBeccar/ODMexamen/refs/heads/main/generar_data/impacto_medida.csv")
impacto_real = pd.read_csv(url2)

In [ ]:
impactos = impacto_real.merge(corta_impacto_i, on='especialidad')
impactos.columns
# renombrar cols: N_acotado y E_acotado son los que quedaron en ODM

Index(['especialidad', 'postulantes', 'N_total', 'E_total', 'cargos',
       'impacto_total', 'impacto_acotado', 'N_acotado', 'E_acotado',
       'N_acotado_R', 'E_acotado_R', 'impacto_total_I', 'impacto_acotado_I',
       'N_acotado_I', 'E_acotado_I'],
      dtype='object')

In [ ]:
# Y los nuevos nombres de las columnas que quieres son
nuevas_columnas = ['especialidad', 'postulantes', 'N_inscriptos', 'E_inscriptos', 'cargos',
       'impacto_total', 'impacto_acotado', 'N_Crudo', 'E_Crudo',
       'N_Real', 'E_Real', 'impacto_total_I', 'impacto_acotado_I',
       'N_Imaginario', 'E_Imaginario']

# Crear el diccionario para el renombramiento
mapeo_columnas = dict(zip(impactos.columns, nuevas_columnas))

# Renombrar las columnas del DataFrame
impactos_renombrado = impactos.rename(columns=mapeo_columnas)

# Opcional: para renombrar en el mismo DataFrame (inplace=True)
# impactos.rename(columns=mapeo_columnas, inplace=True)

se estima que entrarian mas Extranjeros en el rango de oferta en el escenario acotado crudo > imaginario > rea

In [ ]:
impactos_renombrado.head()

,especialidad,postulantes,N_inscriptos,E_inscriptos,cargos,impacto_total,impacto_acotado,N_Crudo,E_Crudo,N_Real,E_Real,impacto_total_I,impacto_acotado_I,N_Imaginario,E_Imaginario
0,PEDIATRIA Y PEDIATRICAS ARTICULADAS,470,337,133,428,67.03%,72.12%,329,99,335,94,34.28%,36.13%,332,96
1,CLINICA MEDICA,285,211,74,267,58.62%,61.24%,207,60,210,58,26.62%,27.59%,208,59
2,NEUROCIRUGIA,170,120,50,23,65.57%,100.00%,21,2,23,0,32.12%,81.82%,22,1
3,CIRUGIA INFANTIL (CIRUGIA PEDIATRICA),54,47,7,8,69.35%,0.00%,8,0,8,0,46.77%,0.00%,8,0
4,TOCOGINECOLOGIA,476,373,103,170,68.96%,100.00%,158,12,171,0,34.41%,54.70%,165,5


In [ ]:
#impactos_renombrado.to_csv('impactos_real_imaginario.csv', index=False)
# subido a Github

## GRAFICOS

## especialidades

In [ ]:
# Clasificar especialidades según el número de postulantes
esp_menor151 = esp_oferta[esp_oferta['postulantes'] < 151].copy()
esp_chicas = esp_menor151['ESPECIALIDAD'].unique().tolist()

esp_151 = esp_oferta[esp_oferta['postulantes'] >= 151].copy()
esp_grandes = esp_151['ESPECIALIDAD'].unique().tolist()


# graficar todos los postulantes

# **Correr la función chicas para esp_chicas**
for esp_chica in esp_chicas:
    row = impactos.loc[impactos['especialidad'] == esp_chica].iloc[0]
    indicador_total = float(str(row['impacto_total']).replace('%', ''))
    chicas(esp_chica, indicador_total)

# **Correr la función grandes para esp_grandes**
for esp_grande in esp_grandes:
    if esp_grande == 'TOTAL':
        continue
    row = impactos.loc[impactos['especialidad'] == esp_grande].iloc[0]
    indicador_total = float(str(row['impacto_total']).replace('%', ''))
    grandes(esp_grande, indicador_total)

In [ ]:
# 2. **Graficar todas las especialidades con la función triple_acotada**
#    Itera sobre la tabla de impactos y llama a la función triple_acotada
for index, row in impactos.iterrows():
    especialidad = row['especialidad']
    if especialidad == 'TOTAL':
        continue
    indicador_acotado_str = row['impacto_acotado']
    if indicador_acotado_str != 'N/A':
        indicador_acotado = float(indicador_acotado_str.replace('%', ''))
        triple_acotada(especialidad, indicador_acotado)

## Globales

In [ ]:
# Gráfico global completo
graficar_global_i(df, esp_oferta)

# Gráfico global acotado por oferta
graficar_global_acotado_i(df, esp_oferta)